In [1]:
##### EDS biasing using HTF #####
## Step 1: Build training graph. Here the collective variable in the EDS method is center of mass##
## Step 2: Set up Hoomd-Sim of arbitrary particle 'A' ##

In [ ]:
#### Build training graph ####
#saved as com_build.py in this example
import hoomd.htf as htf
import sys
import tensorflow as tf

def make_eds_graph(N, NN, directory,set_pt):
    #N= Number of atoms in the system, NN=Number of nearest neighbors, directory=directory to save the results, set_pt=set point in EDS method
    '''Currently only computes running mean'''

    graph =htf.graph_builder(NN,output_forces=True)
    com = tf.reduce_mean(graph.positions[:, :2], 0) #calculate center of mass
    rs = graph.safe_norm(tf.math.subtract(graph.positions[:, :2], com), axis=1) #calculate distance of each atom from center of mass
    real_cv = tf.reduce_mean(rs) #calculate the average distance from center of mass. This is the collective variable (CV)
    cv_run=graph.running_mean(tensor=real_cv,name='cv_run') #calculates the running mean of the CV
    eds_alpha = htf.eds_bias(real_cv, set_point=set_pt, period=300,learning_rate=5.0) #Calculates the EDS alpha value every 300 steps. See htf/utils.py for more information
    eds_energy = eds_alpha * real_cv #computes EDS energy
    eds_forces = graph.compute_forces(eds_energy) #computes EDS forces
    tf.summary.scalar('collective variable',cv_run) #write a scalar summary of CV for tensorboard
    printer1=tf.Print(cv_run,[cv_run],message='CV')
    printer2=tf.Print(eds_alpha,[eds_alpha],message='a')
    graph.save(model_directory=directory,out_nodes=[[printer1,100],[printer2,100]],force_tensor=eds_forces, virial=None)


In [ ]:
### Hoomd-Sim code ###
import hoomd
import hoomd.md
import hoomd.htf
import tensorflow as tf
import sys
import com_build #import the graph building script

model_dir = '/home/EDS/methanol_codes/methanol_results/'

com_build.make_eds_graph(64, 30, model_dir, 4.0)

with hoomd.htf.tfcompute(model_dir,write_tensorboard=True) as tfcompute: #write data to tensorboard in realtime
    hoomd.context.initialize()
    N = 64
    NN = 30
    rcut = 6.0 #cut off radius: must be less than the box size
    system = hoomd.init.create_lattice(unitcell=hoomd.lattice.sq(a=2.0),n=[8, 8]) #initialize the lattice
    hoomd.md.update.enforce2d()
    nlist = hoomd.md.nlist.cell(check_period=1)
    lj = hoomd.md.pair.lj(rcut, nlist) #enable lj pair potential
    lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0) #set lj coefficients
    hoomd.md.integrate.mode_standard(dt=0.005)
    hoomd.md.integrate.nvt(kT=1.0, tau=0.5,group=hoomd.group.all()) # set up NVT simulation
    hoomd.analyze.log(filename='eds.log',quantities=['momentum', 'temperature', 'time'],period=100, overwrite=True) #set up output log
    
    #equilibrate
    hoomd.run(3000)

    #simulation                  
    tfcompute.attach(nlist, r_cut=rcut,)
    hoomd.dump.gsd(filename='saveall_running.gsd',overwrite=True, period=100, group=hoomd.group.all(),dynamic=['attribute', 'momentum', 'topology']) #dump in gsd format
    hoomd.run(10000)

print('Training Done')
